In [1]:
import os
os.chdir("drive/MyDrive/fyp/src")

In [2]:
from data import PolygonSentenceReader, Polygon
from modules import NLLLoss, ELBOLoss
from neural_process.models.np import CNP, LNP
from neural_process.models.attnnp import AttnCNP, AttnLNP
from utils import plot_polygon

import datetime
import random
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.distributions.kl import kl_divergence

In [4]:
TRAINING_ITERATIONS = int(2e5)
PLOT_AFTER = int(1e4)
BATCH_SIZE = 64
MAX_CONTEXT_POINTS = 10
MAX_SEQ_LEN = 5 + 4 * 8
torch.manual_seed(0)

criterion = ELBOLoss()

# Instantiate a polygon generator.
# (For example, polygons with between 3 and 8 sides.)
polygon_generator_train = PolygonSentenceReader(
    batch_size=BATCH_SIZE,
    max_num_context=MAX_CONTEXT_POINTS,
    max_seq_len=MAX_SEQ_LEN,
    min_num_sides=3,
    max_num_sides=8,
    center=(5, 5),
    radius=3,
    testing=False,
)

polygon_generator_test = PolygonSentenceReader(
    batch_size=100,
    max_num_context=MAX_CONTEXT_POINTS,
    max_seq_len=MAX_SEQ_LEN,
    min_num_sides=4,
    max_num_sides=4,
    center=(5, 5),
    radius=3,
    testing=True,
)

model = AttnLNP(x_dim=MAX_SEQ_LEN, y_dim=MAX_SEQ_LEN, attention_type="multihead")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=TRAINING_ITERATIONS, eta_min=1e-6
)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = next(model.parameters()).device
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

Total parameters: 477,770


In [5]:
# ----------------------
# Training Loop
# ----------------------

progress_bar = tqdm(range(TRAINING_ITERATIONS + 1), desc="Training", unit="iter")
for it in progress_bar:
    # Generate a batch of polygon completion tasks.
    (
        context_x,
        context_y,
        target_x,
        target_y,
        tokens,
        true_poly,
        max_seq_len,
        num_context,
        context_masks,
    ) = polygon_generator_train.generate_polygon_batch_few_shot_masked_completion_task()

    context_x = context_x.to(device)
    context_y = context_y.to(device)
    target_x = target_x.to(device)
    target_y = target_y.to(device)
    context_masks = context_masks.to(device)

    optimizer.zero_grad()
    # Forward pass: the NP model expects context_x, context_y, target_x, target_y.
    dist, z, q_zc, q_zct = model(context_x, context_y, target_x, target_y)
    loss = criterion(dist, q_zct, q_zc, target_y, mask=context_masks)
    loss.backward()

    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()

    if it % 1000 == 0:
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        progress_bar.set_postfix({"Iteration": it, "Train Loss": f"{loss.item():.4f}", "LR": f"{scheduler.get_last_lr()[0]:.6f}"})
        print(f"{now}: Iteration: {it}, Train Loss: {loss.item():.4f}")

    # ----------------------
    # Evaluation and Plotting
    # ----------------------
    if it % PLOT_AFTER == 0:
        # For plotting, we generate a single polygon sample.
        (
            context_x_eval,
            context_y_eval,
            target_x_eval,
            target_y_eval,
            tokens_eval,
            true_poly_eval,
            max_seq_len_eval,
            num_context_eval,
            context_masks_eval,
        ) = polygon_generator_test.generate_polygon_batch_few_shot_masked_completion_task()

        context_x_eval = context_x_eval.to(device)
        context_y_eval = context_y_eval.to(device)
        target_x_eval = target_x_eval.to(device)
        target_y_eval = target_y_eval.to(device)
        context_masks_eval = context_masks_eval.to(device)

        # Forward pass through the model.
        test_dist, test_z, test_q_zc, test_q_zct = model(
            context_x_eval, context_y_eval, target_x_eval, target_y_eval
        )
        loss_eval = criterion(dist, q_zct, q_zc, target_y, mask=context_masks)

        # Get NP predictions on target tokens (taking the mean of the predictive distribution).
        pred_target = test_dist.mean  # shape: [n_z, batch, n_target, y_dim]

        # For simplicity, we average over latent samples and batch dimension.
        pred_target_avg = pred_target.mean(0)[0]  # shape: [n_target, y_dim]
        # print("pred_target_avg shape: ", pred_target_avg.shape)
        pred_target_avg = pred_target_avg[0]
        true_polygon = true_poly_eval[0]
        true_polygon_tokenised = true_polygon.to_tokenised()
        pred_target_avg = pred_target_avg[: len(true_polygon_tokenised)].tolist()
        # print("pred_target_avg shape after squeeze: ", pred_target_avg.shape)

        # Squeeze out the batch‐and‐context dims to get a 1D mask
        mask_1d = context_masks_eval.squeeze(1)[0].cpu().tolist()
        mask_1d = mask_1d[: len(true_polygon_tokenised)]

        # Construct predicted tokenised form
        predicted_tokens = []
        for t_true, t_pred, m in zip(
            true_polygon_tokenised, pred_target_avg, mask_1d
        ):
            # m == 1: masked token
            predicted_tokens.append(t_pred if m == 1 else t_true)

        # Reconstruct polygons from tokens.
        try:
            predicted_polygon = Polygon.from_tokenised(predicted_tokens, int(true_polygon_tokenised[0]))
        except Exception as e:
            print("Error reconstructing polygon:", e)
            predicted_polygon = None

        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{now}: Iteration: {it}, Test Loss: {loss_eval.item():.4f}")
        print("True Polygon: ", true_polygon)
        if predicted_polygon is not None:
            print("Predicted Polygon Vertices: ", predicted_polygon)
        else:
            print("Predicted Polygon could not be reconstructed.")

        # Plot true and predicted polygons.
        # plot_polygon(true_polygon.vertices, title="True Polygon")
        # if predicted_polygon is not None:
        #     try:
        #         plot_polygon(predicted_polygon.vertices, title="Predicted Polygon")
        #     except Exception as e:
        #         print("Invalid polygon for plotting:", e)

Training:   0%|          | 3/200001 [00:01<19:19:43,  2.87iter/s, Iteration=0, Train Loss=624617984.0000, LR=0.000100]

2025-04-27 19:37:40: Iteration: 0, Train Loss: 624617984.0000
2025-04-27 19:37:40: Iteration: 0, Test Loss: 624617984.0000
True Polygon:  Polygon(n=4, vertices=[(6.88, 7.34), (6.86, 7.36), (4.0, 7.83), (7.92, 4.29)], lengths=[0.028284271247461926, 2.89836160614924, 5.281855734493323, 3.222436966024316], angles=[153.82853483662774, 144.33232633462882, 32.75164224063834, 29.08749658810514])
Predicted Polygon Vertices:  Polygon(n=4, vertices=[(6.88, 7.34), (6.86, 7.36), (4.0, 7.83), (7.92, 4.29)], lengths=[0.028284271247461926, 2.89836160614924, 5.281855734493323, 3.222436966024316], angles=[-8.852704048156738, -9.074834823608398, 1.0372321605682373, 5.379190921783447])


Training:   1%|          | 1006/200001 [00:44<2:18:31, 23.94iter/s, Iteration=1000, Train Loss=129011.7734, LR=0.000100]

2025-04-27 19:38:23: Iteration: 1000, Train Loss: 129011.7734


Training:   1%|          | 2006/200001 [01:27<2:13:09, 24.78iter/s, Iteration=2000, Train Loss=86186.3359, LR=0.000100]

...

2025-04-27 21:59:24: Iteration: 198000, Train Loss: 14.3013


Training: 100%|█████████▉| 199005/200001 [2:22:28<00:39, 25.12iter/s, Iteration=199000, Train Loss=9.5791, LR=0.000001]

2025-04-27 22:00:07: Iteration: 199000, Train Loss: 9.5791


Training: 100%|██████████| 200001/200001 [2:23:11<00:00, 23.28iter/s, Iteration=2e+5, Train Loss=36.4303, LR=0.000001]

2025-04-27 22:00:50: Iteration: 200000, Train Loss: 36.4303
2025-04-27 22:00:50: Iteration: 200000, Test Loss: 36.4303
True Polygon:  Polygon(n=4, vertices=[(2.95, 7.19), (2.06, 4.42), (6.93, 2.7), (7.14, 2.9)], lengths=[2.9094673051952316, 5.164813646202542, 0.2899999999999998, 5.9966824161364425], angles=[62.13656667525377, 91.64014880848067, 116.94483908310006, 89.27844543316552])
Predicted Polygon Vertices:  Polygon(n=4, vertices=[(2.95, 7.19), (2.06, 4.42), (6.93, 2.7), (7.14, 2.9)], lengths=[2.9094673051952316, 5.164813646202542, 0.2899999999999998, 5.9966824161364425], angles=[55.889442443847656, 78.99274444580078, 22.425039291381836, 79.06021118164062])
